In [10]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

dt = pd.read_csv("TrainSet_1st.csv")
test = pd.read_csv("ValidationSet_2차.csv")
dt.columns = ['F','C','Y']
test.columns = ['F','C','Y']

dt = dt.drop_duplicates(subset = None, keep= 'first', inplace= False, ignore_index= True)
dt.fillna("", inplace=True)
test.fillna("", inplace = True)
dt['New'] = dt['F'] + "\n" + dt['C']
test['New'] = test['F'] + "\n" + test['C']
dt1 = dt[['New', 'Y']]
test1 = test[['New','Y']]

dt_ch_text=dt1[dt1['New'].str.contains('no specific interval change compare to the latest MR brain',
                                       na=False)]
a = dt_ch_text[dt_ch_text['Y'] != 0].index
dt_ch1_text=dt1[dt1['New'].str.contains('No evidence of acute infarction', na=False)]
b= dt_ch1_text[dt_ch1_text['Y'] != 0].index
dt1.drop(a, axis=0, inplace=True)
dt1.drop(b, axis=0, inplace=True)

def howmax(text):
  li = re.findall("\n[0-9]+[.] ", re.sub(" *\n *", "\n", text))
  li = [re.sub("[^\d+]", "", i) for i in li]

  if len(li) != 0:
    li = [int(i) for i in li]
    res = max(li)
    return res

num = max(dt1['New'].map(str).apply(lambda x:howmax(x)))

dt1 = dt1.replace('\r', ' ', regex=True)
dt1 = dt1.replace('Clinical information','', regex=True)
dt1 = dt1.replace('CI','', regex=True)
dt1 = dt1.replace('조영증강은 시행하지 않았음', '', regex=True)

num1 = max(test1['New'].map(str).apply(lambda x:howmax(x)))

test1 = test1.replace('\r', ' ', regex=True)
test1 = test1.replace('Clinical information','', regex=True)
test1 = test1.replace('CI','', regex=True)
test1 = test1.replace('조영증강은 시행하지 않았음', '', regex=True)

def clean_text(text):

  text_rmv = re.sub("[^a-zA-Z가-힣0-9\n +/.]", " ", text)
  text_rmv = re.sub("\n+", "\n", text_rmv)
  text_rmv = re.sub(" +", " ", text_rmv)
  text_rmv = ' '.join(text_rmv.split())

  return text_rmv

dt1['New'] = dt1['New'].map(str).apply(lambda x: clean_text(x))
test1['New'] = test1['New'].map(str).apply(lambda x: clean_text(x))

emlist = []
def delnum(bb):
  for i in range(1,int(bb)+1):
    soot = str(i) + str('. ')
    emlist.append(soot)
  return emlist

final_num_list = delnum(num)
final_num_list

def seee(daataa, f_num_l):

  for i in daataa.index.tolist():

    for k in f_num_l:
      daataa[i] = daataa[i].replace(k, ' ')
    
  return daataa

dt1['New'] = seee(dt1['New'], final_num_list)

X = dt1['New']
y = dt1['Y']

test_X = test1['New']
test_y = test1['Y']

tfidf = TfidfVectorizer(ngram_range=(1,3),min_df=2, max_df=0.9)
tfidf.fit(X)

X_train_tfidf = tfidf.transform(X)
X_test_tfidf = tfidf.transform(test_X)

lr = LogisticRegression(C=10, max_iter=80)
lr.fit(X_train_tfidf, y)
pred = lr.predict(X_test_tfidf)
pred_probs = lr.predict_proba(X_test_tfidf)[:,1]

auroc = roc_auc_score(test_y, pred_probs)

print('C-Statistics: {}'.format(auroc))

/var/folders/z4/1fybx5lj43j2j5fzhmjsjx_c0000gn/T/ipykernel_9449/1794254240.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt1.drop(a, axis=0, inplace=True)
/var/folders/z4/1fybx5lj43j2j5fzhmjsjx_c0000gn/T/ipykernel_9449/1794254240.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt1.drop(b, axis=0, inplace=True)
/var/folders/z4/1fybx5lj43j2j5fzhmjsjx_c0000gn/T/ipykernel_9449/1794254240.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

C-Statistics: 0.9953373123530476


/Users/ihoon/opt/anaconda3/envs/bhoon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
